<a href="https://colab.research.google.com/github/mracy/Robust-Text-Extraction-from-Painted-Screenshots-Using-Adaptive-Inpainting-and-OCR/blob/main/Robust_Text_Extraction_from_Painted_Screenshots_Using_Adaptive_Inpainting_and_OCR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pytesseract

In [ ]:
!apt install tesseract-ocr


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.


In [ ]:
!pip install --upgrade opencv-python

In [ ]:
import cv2
import pytesseract
import numpy as np
from google.colab.patches import cv2_imshow  # Import for image display

def read_painted_text(image_path):
  """
  Reads text from a screenshot, handling painted areas using various techniques.

  Args:
      image_path (str): Path to the screenshot image.

  Returns:
      str: Extracted text from the screenshot.
  """

  # Load the image
  image = cv2.imread(image_path)

  # Preprocessing for Noise Reduction (Optional)
  # Apply techniques like blurring or bilateral filtering to reduce noise
  # that may hinder text recognition, especially before inpainting.
  # Uncomment the following lines if needed:
  # gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
  # blurred = cv2.bilateralFilter(gray, 9, 75, 75)

  # Adaptive Inpainting with Color Thresholding (if applicable)
  try:
    # Convert to HSV colorspace for better color-based segmentation
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    # Define color ranges for painted areas (adjust thresholds as needed)
    lower_paint = np.array([0, 100, 100])
    upper_paint = np.array([10, 255, 255])
    # Create a mask using color thresholding
    mask = cv2.inRange(hsv, lower_paint, upper_paint)
    # Expand the mask slightly to capture potential edges around painted areas
    kernel = np.ones((3, 3), np.uint8)
    mask = cv2.dilate(mask, kernel, iterations=1)
  except:
    # Fallback to manual coordinates if thresholding fails
    print("Color thresholding failed. Using manual coordinates.")
    painted_area_coordinates = [(100, 200), (300, 400), (500, 600)]  # Replace with actual coordinates
    mask = np.zeros(image.shape[:2], dtype=np.uint8)
    for (x, y) in painted_area_coordinates:
      cv2.circle(mask, (x, y), 10, (255), -1)

  # Inpainting
  try:
    # Attempt inpainting with TELEA flag (OpenCV 4.2.0 or later recommended)
    inpaint_mask = cv2.inpaint(image, mask, inpaintRadius=3, flags=cv2.INPAINT_TELEA_GMCU)
  except:
    print("INPAINT_TELEA failed. Using fastNlMeansDenoisingColored")
    # Fallback to alternative inpainting method
    inpaint_mask = cv2.fastNlMeansDenoisingColored(image, None, 10, 10, 7, 21)

  # Text Recognition with Tesseract Configuration
  text = pytesseract.image_to_string(inpaint_mask, config='--psm 6')  # Adjust config as needed

  return text

# Example usage
image_path = '/content/Screenshot 2024-05-04 at 23.51.04.png'  # Replace with your image path
extracted_text = read_painted_text(image_path)
print("Extracted Text:")
print(extracted_text)

# Display the inpainted image (optional)
# cv2_imshow(inpaint_mask)
# cv2.waitKey(0)
# cv2.destroyAllWindows()


INPAINT_TELEA failed. Using fastNlMeansDenoisingColored
Extracted Text:
no
Veter Mostar v NK Posusje Fri 3 Mey 17:00
Zetjeznicar v FK Turtle City Fri 3 Mey ore)
ry A
or ——

